In [57]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [58]:
dtrain = pd.read_csv('data/cleaned_train.csv')
dtest = pd.read_csv('data/cleaned_test.csv')

In [59]:
# dtrain['ApplicantIncome'] = np.log10(dtrain.ApplicantIncome + 0.1)
# dtest['ApplicantIncome'] =np.log10(dtest.ApplicantIncome + 0.1)

# dtrain['CoapplicantIncome'] = np.log10(dtrain.CoapplicantIncome + 0.1)
# dtest['CoapplicantIncome'] =np.log10(dtest.CoapplicantIncome + 0.1)

# dtrain['LoanAmount'] = np.log10(dtrain.LoanAmount + 0.1)
# dtest['LoanAmount'] =np.log10(dtest.LoanAmount + 0.1)

# dtrain['Loan_Amount_Term'] = np.log10(dtrain.Loan_Amount_Term + 0.1)
# dtest['Loan_Amount_Term'] =np.log10(dtest.Loan_Amount_Term + 0.1)

dtrain['Gender'] = np.log10(dtrain.Gender)
dtest['Gender'] = np.log10(dtrain.Gender)
# dtrain.

In [60]:
dtrain['Income'] = dtrain.ApplicantIncome + dtrain.CoapplicantIncome
dtest['Income'] = dtest.ApplicantIncome + dtest.CoapplicantIncome

dtrain['MonthlyLoan'] = dtrain.LoanAmount / dtrain.Loan_Amount_Term
dtest['MonthlyLoan'] = dtest.LoanAmount / dtest.Loan_Amount_Term

dtrain['Income'] = np.log10(dtrain.Income)
dtest['Income'] = np.log10(dtest.Income)

# dtrain['Income'] = (dtrain.Income * dtrain.Property_Area) + dtrain.Education
# dtest['Income'] = (dtest.Income * dtest.Property_Area) + dtest.Education

ls = dtrain.Loan_Status

# dtrain.drop(['Loan_Status'], axis=1, inplace=True)

dtrain.drop(['Loan_Status', 
             'ApplicantIncome',
             'CoapplicantIncome',
             'LoanAmount',
             'Loan_Amount_Term',
            ], axis=1, inplace=True)

dtest.drop(['ApplicantIncome',
             'CoapplicantIncome',
             'LoanAmount',
             'Loan_Amount_Term'
            ], axis=1, inplace=True)


dtrain['Loan_Status'] = ls


In [61]:
ids = dtest.Loan_ID
dtrain = dtrain.iloc[:, 2:]
dtest = dtest.iloc[:, 2:]

In [62]:
dtrain.head()

,Gender,Married,Dependents,Education,Self_Employed,Credit_History,Property_Area,Income,MonthlyLoan,Loan_Status
0,3.121569,0,0,0.708333,1,1.0,0.658416,3.767082,0.355556,1
1,3.121569,2,1,0.708333,1,1.0,0.614525,3.784689,0.355556,0
2,3.121569,2,0,0.708333,2,1.0,0.658416,3.477121,0.183333,1
3,3.121569,2,0,0.611940,1,1.0,0.658416,3.693815,0.333333,1
4,3.121569,0,0,0.708333,1,1.0,0.658416,3.778151,0.391667,1


In [63]:
labels = np.array(dtrain.iloc[:, -1]) # features 
features = np.array(dtrain.iloc[:, 0:-1]) # labels
test = np.array(dtest)

In [64]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
features_std = scalar.fit_transform(features)
test_std = scalar.fit_transform(test)
X_train, X_test, y_train, y_test = train_test_split(features_std, labels, test_size=0.33, random_state=42)

In [65]:
# Creating XBG classifier
from xgboost.sklearn import XGBClassifier
xgb = XGBClassifier(learning_rate =0.03, 
                    n_estimators=3000, 
                    max_depth=8, 
                    min_child_weight=6,
                    gamma=0.1,
                    subsample=0.95,
                    colsample_bytree=0.95, 
                    reg_alpha=2,
                    objective= 'binary:logistic', 
                    nthread=-1,
                    scale_pos_weight=1, 
                    seed=27)

In [66]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import StratifiedKFold
forest = RandomForestClassifier(
    n_estimators = 3000, 
    criterion='entropy', 
    oob_score=True, 
    random_state=42,
    n_jobs=-1)

# forest = RandomForestClassifier(n_jobs=-1, n_estimators=200, 
#                              oob_score = True, max_depth=3, max_features='auto')
# param_grid = { 
#     'n_estimators': [200, 700],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'max_depth': [5, 3, 1]
# }
# # skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
# CV_rfc = GridSearchCV(estimator=forest, param_grid=param_grid, cv=4, scoring='accuracy', verbose=1)
# CV_rfc.fit(features, labels)
# print(CV_rfc.best_params_)

In [67]:
# Creating a KNN classifier
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)

In [68]:
# Creating AdaBoost Classifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(criterion='gini',max_depth=1)
ada = AdaBoostClassifier(base_estimator=tree, n_estimators=3000, learning_rate=0.03, random_state=42)

In [69]:
# Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(
    n_estimators=3000,
    random_state=42)

In [70]:
# Creating Enesemble Now with these 3 classifiers
from sklearn.ensemble import VotingClassifier
eclf = VotingClassifier(estimators=[('forest', forest), ('gbc', xgb), ('adaboost', ada)], 
                        voting='hard')

In [78]:
eclf.fit(features, labels)

VotingClassifier(estimators=[('forest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
           ...e,
            splitter='best'),
          learning_rate=0.03, n_estimators=3000, random_state=42))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [79]:
pred = eclf.predict(test)
# test_std.shape

In [76]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, pred, normalize=True) * 100)

78.7878787879


In [80]:
submission = pd.DataFrame({'Loan_ID': ids, 'Loan_Status': pred})

In [81]:
submission['Loan_Status'] = submission.Loan_Status.map({0 : 'N', 1: 'Y'})

In [82]:
submission.to_csv('submission_.csv', index=False)

In [1218]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
data_map = {}
for train_index, test_index in skf.split(features, labels):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    eclf.fit(X_train, y_train)
    pred = eclf.predict(X_test)
    acc = accuracy_score(y_test, pred, normalize=True) * 100
    data_map[str(acc)] = (X_train, X_test, y_train, y_test)

In [1222]:
X_train, X_test, y_train, y_test = data_map['89.6551724138']

In [1223]:
eclf.fit(X_train, y_train)
pred = eclf.predict(X_test)
accuracy_score(y_test, pred, normalize=True) * 100

89.65517241379311

In [739]:
xgb = xgb1.fit(features, labels)

In [740]:
pred = xgb.predict(test)

In [742]:
pred

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1,

In [803]:
dtest

,Gender,Married,Dependents,Education,Self_Employed,Credit_History,Property_Area,Income,MonthlyLoan
0,0.693252,1,0,0,1,1.0,0.658416,2.757404,0.798689
1,0.693252,1,1,0,1,1.0,0.658416,6.664121,0.821740
2,0.693252,1,2,0,1,1.0,0.658416,6.954275,0.906842
3,0.693252,1,2,0,1,1.0,0.658416,6.775110,0.782513
4,0.693252,0,0,0,1,1.0,0.658416,2.515357,0.740351
5,0.693252,1,0,0,2,1.0,0.658416,6.869771,0.853587
6,0.669643,0,1,0,1,1.0,0.768240,2.347545,0.692995
7,0.693252,1,2,0,1,0.0,0.614525,2.588955,0.847908
8,0.693252,1,2,0,0,1.0,0.658416,3.134595,1.028113
9,0.693252,0,0,0,1,1.0,0.768240,6.760459,0.817649


In [1107]:
k = dtrain.iloc[:, 2:]

In [1129]:
dtrain.corr()

,Gender,Married,Dependents,Self_Employed,Credit_History,Income,MonthlyLoan,Loan_Status
Gender,1.000000,0.238519,0.107555,-0.071497,0.048585,0.122770,0.100237,0.046585
Married,0.238519,1.000000,0.266627,-0.001205,0.019704,0.351886,0.214292,0.086811
Dependents,0.107555,0.266627,1.000000,0.036952,-0.025719,-0.006130,0.131965,0.000928
Self_Employed,-0.071497,-0.001205,0.036952,1.000000,-0.045246,-0.067701,0.089035,-0.027470
Credit_History,0.048585,0.019704,-0.025719,-0.045246,1.000000,0.030244,-0.018154,0.558416
Income,0.122770,0.351886,-0.006130,-0.067701,0.030244,1.000000,0.218821,0.121065
MonthlyLoan,0.100237,0.214292,0.131965,0.089035,-0.018154,0.218821,1.000000,-0.013375
Loan_Status,0.046585,0.086811,0.000928,-0.027470,0.558416,0.121065,-0.013375,1.000000


In [1166]:
features.shape

(575, 7)

In [1168]:
test.shape

(367, 7)

In [1421]:
dtrain

,Gender,Married,Dependents,Education,Self_Employed,Credit_History,Property_Area,Income,MonthlyLoan,Loan_Status
0,3.121569,0,0,0.708333,1,1.0,0.658416,5849.0,0.355556,1
1,3.121569,2,1,0.708333,1,1.0,0.614525,6091.0,0.355556,0
2,3.121569,2,0,0.708333,2,1.0,0.658416,3000.0,0.183333,1
3,3.121569,2,0,0.611940,1,1.0,0.658416,4941.0,0.333333,1
4,3.121569,0,0,0.708333,1,1.0,0.658416,6000.0,0.391667,1
5,3.121569,2,2,0.708333,2,1.0,0.658416,9613.0,0.741667,1
6,3.121569,2,0,0.611940,1,1.0,0.658416,3849.0,0.263889,1
7,3.121569,2,6,0.708333,1,0.0,0.768240,5540.0,0.438889,0
8,3.121569,2,2,0.708333,1,1.0,0.658416,5532.0,0.466667,1
9,3.121569,2,2,0.708333,1,1.0,0.658416,3900.0,0.194444,1
